In [1]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Вспомогательные функции
def check_for_none(x):
    if x is None:
        return ''
    else:
        return x.text

# Parsing

## Avito

In [3]:
main_url = 'https://www.avito.ru'

In [59]:
final_data = []

for i in range(1,11):
    add_url = '/moskva_i_mo/avtomobili?p='+str(i)
    pred_ap = {}
    full_ap = []

    # Короткий текст объявления в шаблоне поиска
    url = main_url + add_url
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    ap = soup.findAll('div', {'class': 'snippet-horizontal'})
    pred_ap['main_title'] = check_for_none(ap[0].find('div', {'class': 'snippet-title-row'}))
    pred_ap['price_comment'] = check_for_none(ap[0].find('span', {'class': 'snippet-tag'}))
    pred_ap['price'] = check_for_none(ap[0].find('span', {'class': 'snippet-price'}))
    pred_ap['specific-params'] = check_for_none(ap[0].find('div', {'class': 'specific-params_block'}))
    pred_ap['address'] = check_for_none(ap[0].find('div', {'class': 'item-address'}))

    ########################################################################
    # Полный текст внутри объявления
    full_app_dict = {}

    url = main_url + ap[0].find('a', {'class': 'snippet-link'}, href=True)['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    print(url)
    fap = soup.find('div', {'class': 'item-params'})
    li = fap.findAll('li', {'class':'item-params-list-item'})
    for l in li:
        full_app_dict[l.text.split(':')[0].strip()] = l.text.split(':')[1].strip()
    full_app_dict['address'] = soup.find('div', {'class':'item-address'}).text
    full_app_dict['full_text_app'] = soup.find('div', class_=re.compile('item-description')).text

    ########################################################################
    # Дополенная информация об объекте
    adv_info = {}

    fap_adv = soup.find('ul', class_=re.compile('advanced-params'))
    if fap_adv is not None:
        li = fap_adv.findAll('li', {'class': 'advanced-params-param'})
        for l in li:
            param_list = []
            title = l.find('div', {'class': 'advanced-params-param-title'}).text
            item_list = l.find('ul', {'class': 'advanced-params-param-list'})  
            for il in item_list.findAll('li', {'class': 'advanced-params-param-item'}):
                    param_list.append(il.text)    
            adv_info[title] = param_list

    ########################################################################
    #Дополнительная информация по цене в объявлении  
    add_price = {}

    js = soup.find('div', class_ = re.compile('js-ssr'))
    if js is not None:    
        add_info = json.loads(js['data-props'])
        add_price['title'] = add_info['title']
        add_price['subtitle'] = add_info['subtitle']
        add_price['priceType'] = add_info['priceType']
        add_price['marketPrice'] = add_info['marketPrice']
        add_price['isVisibleForUser'] = add_info['isVisibleForUser']
        add_price['price'] = add_info['price']
    else:
        add_price['title'] = ''
        add_price['subtitle'] = ''
        add_price['priceType'] = ''
        add_price['marketPrice'] = ''
        add_price['isVisibleForUser'] = ''
        add_price['price'] = ''

    ########################################################################
    # Технические характеритики модели
    spec_table = {}
    url = main_url + soup.find('a', {'class': 'item-view-specification-link'}, href=True)['href']
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    table = soup.findAll('table', class_=re.compile('specification-table'))
    for tab in table:
        tr = tab.findAll('tr')
        for t in tr:
            td = t.findAll('td')
            spec_table[td[0].text] = td[1].text

    # Собираем все данныe в один DataFrame
    final_data.append({**spec_table, **add_price, **pred_ap, **adv_info, **full_app_dict})

https://www.avito.ru/moskovskaya_oblast_krasnogorsk/avtomobili/datsun_mi-do_2018_1887856696
https://www.avito.ru/autocatalog/datsun/mi-do/i-2015n-v_903/khetchbek/344402
https://www.avito.ru/moskva/avtomobili/volkswagen_polo_2010_1887988882
https://www.avito.ru/autocatalog/volkswagen/polo/v-20092015/sedan/343204
https://www.avito.ru/moskva/avtomobili/porsche_cayenne_2018_1881613943
https://www.avito.ru/autocatalog/porsche/cayenne/ii-restayling-20142018/vnedorozhnik/359896
https://www.avito.ru/vidnoe/avtomobili/land_rover_discovery_2009_1854446232
https://www.avito.ru/autocatalog/land_rover/discovery/iv-20092013/vnedorozhnik/360464
https://www.avito.ru/moskva/avtomobili/bmw_x5_2010_1908281451
https://www.avito.ru/autocatalog/bmw/x5/e70-restayling-20102013/vnedorozhnik/362752
https://www.avito.ru/moskva/avtomobili/ford_maverick_2006_1863030793
https://www.avito.ru/autocatalog/ford/maverick/ii-20002007/vnedorozhnik/359426
https://www.avito.ru/moskva/avtomobili/hyundai_i30_n_2019_1865305802

In [60]:
avito_data = pd.DataFrame(final_data)
avito_data.head()

,VIN или номер кузова,address,full_text_app,isVisibleForUser,main_title,marketPrice,price,priceType,price_comment,specific-params,...,Тип кузова,Управление климатом,Усилитель руля,Фары,Цвет,Шины и диски,"Ширина (с зеркалами), мм",Экологический класс,Электропривод,Электростеклоподъемники
0,Z8NF*************,"\n\n Московская область, г.о. Красногорск, д. ...",\nКомплектация trust 2.Машина в хорошем состо...,,"Datsun mi-DO, 2018",,\n 450 000 ₽\n,,,"\n 65 722 км, 1.6 MT (87 л.с.), хетчбэк, пере...",...,хетчбэк,NaN,NaN,NaN,оранжевый,NaN,NaN,Euro 5,NaN,NaN
1,XW8Z*************,"\n\n Москва, Северо-Западный административный ...","\nОсмотр м""мякинино"" 15 минут пешком от метро...",,"Volkswagen Polo, 2010",,\n 349 000 ₽\n,,,"\n 115 000 км, 1.6 AT (105 л.с.), седан, пере...",...,седан,NaN,NaN,NaN,синий,NaN,NaN,Euro 4,NaN,NaN
2,WP1Z*************,"\n\n Москва, Болотниковская ул., 42к1\n Нахимо...","\nАвто в отличном состоянии, хорошая комплект...",True,"Porsche Cayenne, 2018Отличная цена",4 083 300 ₽,\n 3 500 000 ₽\n,low,Отличная цена,"\n 21 000 км, 3.0 AT (245 л.с.), внедорожник,...",...,внедорожник,NaN,NaN,NaN,чёрный,NaN,NaN,Euro 5,NaN,NaN
3,SALL*************,"\n\n Московская область, Ленинский г.о., Видно...",\nПродаю СВОЙ Discovery.Цена указана на штатн...,True,"Land Rover Discovery, 2009Хорошая цена",1 179 300 ₽,\n 1 170 000 ₽\n,normal,Хорошая цена,"\n 152 000 км, 3.0 AT (245 л.с.), внедорожник...",...,внедорожник,NaN,NaN,NaN,синий,NaN,NaN,Euro 4,NaN,NaN
4,WBAZ*************,"\n\n Москва, Высокая ул.\n Коломенская, 500\t ...",\nОбщее:Без ДТППробег реальный Один владелец ...,True,"BMW X5, 2010Хорошая цена",1 369 900 ₽,\n 1 400 000 ₽\n,normal,Хорошая цена,"\n 102 000 км, 3.0 AT (306 л.с.), внедорожник...",...,внедорожник,[Климат многозонный],[Электрогидроусилитель],"[Ксеноновые, Противотуманные, Омыватели фар, А...",серый,"[19"", Зимние шины в комплекте]",NaN,Euro 4,"[Передних сидений, Зеркал, Рулевой колонки, Ск...",[Передние и задние]
